# AutoML Classifier

This is a component that trains an AutoML Classifier model using [auto-sklearn](https://github.com/automl/auto-sklearn). 
<br>
auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator.

This notebook shows:
- how to use the [SDK](https://platiagro.github.io/sdk/) to load datasets, save models and other artifacts.
- how to declare parameters and use them to build reusable components.

## Declare parameters
Components may declare (and use) these default parameters:
- dataset
- target
- experiment_id
- operator_id

Use these parameters to load/save datasets, models, metrics, and figures with the help of [PlatIAgro SDK](https://platiagro.github.io/sdk/).

You may also declare custom parameters to set when running an experiment.

In [ ]:
dataset = "iris" #@param {type:"string"}
target = "Species" #@param {type:"string"}
experiment_id = "48f2668a-e31a-4b5a-a91a-28fd649f7adc" #@param {type:"string"}
operator_id = "64401802-2785-4f26-85c8-d7974fc78454" #@param {type:"string"}
duration = 60 #@param {type:"integer", label:"Limite de tempo (em segundos)"}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1).to_numpy()
y = df[target].to_numpy()

## Load metadata about the dataset
For example, below we get the feature type for each column in the dataset. (eg. categorical, numerical, or datetime)

In [ ]:
import numpy as np
from platiagro import stat_dataset
from platiagro.featuretypes import infer_featuretypes

try:
    metadata = stat_dataset(name=dataset)
    featuretypes = metadata["featuretypes"]
except KeyError:
    featuretypes = infer_featuretypes(df)

featuretypes = np.array(featuretypes)

## Replace NaN values
Remove features that all values are NA.<br>
If some values are missing, then use the mean for numerical features, and the mode for categorical features.

In [ ]:
na_free = df.dropna(axis="columns", how="all")
only_na = df.loc[:, ~df.columns.isin(na_free.columns)]

featuretypes = featuretypes[df.columns.isin(na_free.columns)]
df = na_free

In [ ]:
from platiagro.featuretypes import CATEGORICAL, NUMERICAL

numerical_indexes = (featuretypes == NUMERICAL)
numerical_nan_replacement = df.iloc[:, numerical_indexes].mean(axis=0)
df.fillna(numerical_nan_replacement, inplace=True)

categorical_indexes = (featuretypes == CATEGORICAL)
categorical_nan_replacement = df.iloc[:, categorical_indexes].mode(axis=0).iloc[0]
df.fillna(categorical_nan_replacement, inplace=True)

In [ ]:
X = df.drop(target, axis=1).to_numpy()
columns = df.columns.to_numpy()
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Remove datetime features
Datetime columns require separate preprocessing or feature extraction steps.

In [ ]:
from platiagro.featuretypes import DATETIME

datetime_indexes = (featuretypes == DATETIME)
X = X[:, np.where(~datetime_indexes)[0]]
featuretypes = np.delete(featuretypes, np.where(datetime_indexes))

## Encode categorical features

Many machine learning algorithms cannot operate on categorical data directly. They require all input variables and output variables to be numeric.<br>
This means that categorical data must be converted to a numerical form.<br>
The features are converted to ordinal integers. This results in a single column of integers (0 to n_categories - 1) per feature.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

categorical_indexes = (featuretypes == CATEGORICAL)
feature_encoder = OrdinalEncoder()

if np.ma.any(categorical_indexes):
    X[:, categorical_indexes] = feature_encoder.fit_transform(X[:, categorical_indexes])

## Encode target labels

The target labels are converted to ordinal integers with value between 0 and n_classes-1.

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

## Split dataset into train/test splits

Training Dataset: the sample of data used to fit the model.

Test Dataset: the sample of data used to provide an unbiased evaluation of a model fit on the training dataset.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,  train_size=0.7)

## Fit a model using autosklearn.classification.AutoSklearnClassifier

In [ ]:
from autosklearn.classification import AutoSklearnClassifier

estimator = AutoSklearnClassifier(
    time_left_for_this_task=duration,
    per_run_time_limit=duration,
)
estimator.fit(X_train, y_train, feat_type=featuretypes)
estimator.refit(X_train, y_train)

## Measure the performance
The [**Confusion Matrix**](https://en.wikipedia.org/wiki/Confusion_matrix) is a performance measurement for machine learning classification.<br>
It is extremely useful for measuring [Accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision#In_binary_classification), [Recall, Precision, and F-measure](https://en.wikipedia.org/wiki/Precision_and_recall).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

# uses the model to make predictions on the Test Dataset
y_pred = estimator.predict(X_test)

# computes confusion matrix
labels = np.unique(y)
data = confusion_matrix(y_test, y_pred, labels=labels)

# puts matrix in pandas.DataFrame for better format
labels = label_encoder.inverse_transform(labels)
confusion_matrix = pd.DataFrame(data, columns=labels, index=labels)

## Save metrics

Record the metrics used to evaluate the model.<br>
It's a good way to document the experiments, and also help to avoid running the same experiment twice. 

In [ ]:
from platiagro import save_metrics

save_metrics(experiment_id=experiment_id, operator_id=operator_id, confusion_matrix=confusion_matrix)

## Save model

Stores the model artifacts in a object storage.<br>
It will make the model available for future deployments.

In [ ]:
from platiagro import save_model

save_model(experiment_id=experiment_id,
           model={"estimator": estimator,
                  "feature_encoder": feature_encoder,
                  "label_encoder": label_encoder,
                  "columns": columns,
                  "datetime_indexes": datetime_indexes,
                  "categorical_indexes": categorical_indexes,
                  "numerical_nan_replacement": numerical_nan_replacement,
                  "categorical_nan_replacement": categorical_nan_replacement})